# 7.2  Variability of Classification

In [1]:
% matplotlib inline

import numpy as np
import scipy as sp
import math
import scipy.stats as scp
from numpy import linalg as la
from numpy import random as rand
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d


In [60]:
def cross_validation(X, y, L, n_folds=10):
    '''
    Synopsis:
        w_opt, b_opt, lambda_opt = cross_validation(X, Y, L, n_folds=10)
    Arguments:
        X:            2D array of data (features x samples)
        Y:            Vector of true labels (1 x samples)
        L:            List of lambdas to cross validate (1 x #lambdas)
        n_folds:      Number of nested folds
    Output:
        w_opt:        optimal weight vector
        b_opt:        optimal bias
        lambda_opt:   the lambda with the lowest MSE
    '''
    d, n = X.shape
    samples_per_fold = int(n / n_folds)
    
    # set up a container for the resulting mse values (n_folds x #lambdas)
    MSE = np.empty((n_folds, L.shape[0]))
    mse_min = float('inf')
    lambda_opt = _

    for i, l in enumerate(L):
        # loop over all possible lambdas
        # using different permutation of samples
        idx = np.arange(n) # np.random.permutation(n) # np.arange(n)
        for j in range(n_folds):
            # extract one fold for testing
            idx_te = idx[j*samples_per_fold:(j+1)*samples_per_fold]
            # get the train data
            X_tr = np.delete(X, idx_te, axis=1)
            y_tr = np.delete(y, idx_te, axis=1)
            # get the test data
            X_te = X[:,idx_te]
            y_te = y[:,idx_te]
            
            print ("y_test")
            print y_te
            # train the model
            w, b = train(X_tr, y_tr, l)
            # predict the labels
            
            
            y_pred = np.sign(w.T.dot(X_te) - b)
            print y_pred.shape
            y_pred = predict(X_te, w, b)
            print y_pred.shape
        
            # and compute the mse for this lambda and fold

            MSE[j,i] = m_sq_error(y_te, y_pred)
            print MSE[j][i]

        # find the lambda with the lowest MSE
        mse_this = MSE[:,i].mean()
        if(mse_this < mse_min):
            mse_min = mse_this
            lambda_opt = l

    return lambda_opt

In [3]:
def m_sq_error(y, y_pred):
    '''
    Synopsis:
        mse = m_sq_error(y, y_pred)
    Arguments:
        y:        Vector of true labels
        y_pred:   Vector of predicted labels
    Output:
        mse:      The mean squared error
    '''
    return ((y_pred - y)**2).mean()


In [4]:
def train(X, y, lmbda):
    '''
    Synopsis:
        w, b = train(X, y, lmda)
    Arguments:
        X:     2D array of data (features x samples)
        y:     Vector of true labels
        lmda:  The regularisation value
    Output:
        w:     The computed weight vector (features x 1)
        b:     The bias (scalar)
    '''    
    # covariance (d x d)
    C = X.dot(X.T)
    # reg, inv cov (d x d)
    B = la.inv(C + np.identity(C.shape[0]) * lmbda)
    # compute the weights (d x 1)
    w = B.dot(X).dot(y.T)
    # compute the bias b (scalar)
    mu_c1 = X[:,y[0,:]==-1].mean(axis=1, keepdims=True)
    mu_c2 = X[:,y[0,:]==1].mean(axis=1, keepdims=True)
    b = w.T.dot(0.5*(mu_c1 + mu_c2))[0,0]
    return w, b

In [56]:
def predict (X,w,b):
    return np.sign(w.T.dot(X) - b)

In [6]:
def generate_data(n):
    mu_1 = [0,1]
    mu_2 = [1,0]
    var1 =[2,0]
    var2 =[0,2]

    X_c1 = np.append(rand.multivariate_normal(mu_1, [var1,var2], n/2), rand.multivariate_normal(mu_1, [var1,var2], n/2), axis=0)
    X_c2 = np.append(rand.multivariate_normal(mu_2, [var1,var2], n/2), rand.multivariate_normal(mu_2, [var1,var2], n/2), axis=0)
    # create a data matrix of shape 120x3 where colums 0 and 1 are the dimensions and colum 2 being the label
    X = np.vstack((X_c1, X_c2)).T
    y = np.vstack((np.ones((n,1)), np.ones((n,1))-2)).T
    
    return X,y

In [7]:
def calc_error(y,y_test):
    f = (y[0,:]!=y_test[0.:]).sum()
    return (float(f)/(y[0,:].shape[0])) * 100

In [61]:
N = [2,4,6,8,10,20,40,100]
L = 10**np.arange(-4, 4, 0.1)
err_test = np.zeros((8,50))
err_train = np.zeros((8,50))

for i,n in enumerate(N):
    
    # repeat 50 Times
    for j in range(10):
        print j
        X_temp, y_temp = generate_data(n)
        lamda_opt = cross_validation(X_temp,y_temp,L)
        w,b = train(X_temp,y_temp,lamda_opt)
        X_test,y_test = generate_data(1000) 
        y_pred_test = predict(X_test,w,b)
        err_test[i][j] = calc_error(y_pred_test,y_test)
        
        y_pred_train = predict(X_temp,w,b)
        err_train[i][j] = calc_error(y_pred_train,y_temp)
        


0
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(2, 1)
(1, 0)
nan
y_test
[]
(1, 0)
(

TypeError: ufunc 'multiply' did not contain a loop with signature matching types dtype('S32') dtype('S32') dtype('S32')

In [ ]:
mean_test = np.zeros((1,8))
mean_train = np.zeros((1,8))
std_test = np.zeros((1,8))
std_train = np.zeros((1,8))
for k in range (8):
    mean_test[0,k] =  np.mean(err_test[k,:])
    mean_train[0,k] = np.mean(err_train[k,:])
    std_test[0,k] = np.std(err_test[k,:])
    std_train[0,k] = np.std(err_train[k,:])
    
    
plt.plot(np.arange(len(N)),mean_test[0,:])
plt.plot(np.arange(len(N)),mean_train[0,:])

plt.figure()
plt.plot(np.arange(len(N)),std_test[0,:])
plt.plot(np.arange(len(N)),std_train[0,:])

# 7.3 The Binomial Distribution


In [ ]:
from scipy.special import comb

In [ ]:
def prob_mass_binom (k,n,p):
    a = comb(n,k)
    b = (1-p)**(n-k)
    return a*(p**k)*b

In [ ]:
K = [1,3,15,28,50]
N = [2,20,50,80,100]
P = [0.1,0.2,0.3,0.4,0.5]

results = np.zeros((5,5,5))
ck= 0
cn = 0
cp = 0
for k in K:
    cn = 0
    for n in N:
        cp = 0
        for  p in P:
            results[ck][cn][cp] = prob_mass_binom(k,n,p)
            cp = cp+1
        cn = cn+1
    ck = ck+1
    
#for i in range(0,ck):
#    plt.figure()
#    plt.contour(P,N,results[i,:,:])
#    plt.title(K[i])

#fig = plt.figure()
#ax = plt.axes(projection='3d')

for i in range(0,ck):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot_surface(P, N, results[i,:,:], cmap=plt.cm.jet, rstride=1, cstride=1, linewidth=0)
            

## 7.3 b)
